In [0]:
import tensorflow as tf
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [69]:
data = pd.read_csv("clogit_df.csv",encoding='utf-8')
print(data.columns)
#data = data[0:27939]

Index(['url', 'date', 'round', 'name', 'code', 'rank', 'lane', 'sex', 'age',
       'jockey_w', 'rating', 'jockey', 'difference', 'weight', 'dandivi',
       'yeondivi', 's1fr', 'c1r', 'c2r', 'c3r', 'c4r', 'g1fr', 's1f', 'c1',
       'c2', 'c3', 'c4', 'g3f', 'g1f', 'record', 'cure_in_1m', 'weight_diff',
       'raw_weight', 'weight_added', 'distance', 'velocity', 'prev1_rank',
       'prev2_rank', 'prev3_rank', 'prev1_velo', 'prev2_velo', 'prev3_velo',
       'horse_level_num', 'groupid', 'avg_vel', 'adj_vel', 'prev_rank',
       'norating', 'choice', 'choice2', 'choice3', 'sex1', 'sex2'],
      dtype='object')


In [83]:

#data = data.dropna() # 이거로 하면 cost 5자리수 나옴
'''
d2 = data
d3=d2.groupby(['distance'])
d3=[d3.get_group(x) for x in d3.groups]
data=d3[0]
'''

y = data['record']
X = data.loc[:, ['date','code','lane','age','jockey_w','dandivi','yeondivi',
                 'cure_in_1m','weight_diff', 'raw_weight','weight_added',
                 'prev1_velo','prev2_velo','sex1','sex2','distance']]
year=[]
month=[]
for i in range(X.shape[0]):
  word =X['date'][i].split("-")
  year.append(int(word[0])-2010)
  month.append(int(word[1]))
  
X['year'] = year
X['month'] = month
#print(X['year'],X['month'])
X=X.drop(['date'],axis=1)

#prev3은 도움안됨
'''
# sex는 dummy 함수 처리
sex_mod = pd.get_dummies(X['sex'])
X = X.join(sex_mod)
X = X.drop(columns=['sex'])
#print(X.columns)
'''
print(X.columns)
X = X.as_matrix()
print(X.shape)

Index(['code', 'lane', 'age', 'jockey_w', 'dandivi', 'yeondivi', 'cure_in_1m',
       'weight_diff', 'raw_weight', 'weight_added', 'prev1_velo', 'prev2_velo',
       'sex1', 'sex2', 'distance', 'year', 'month'],
      dtype='object')
(56576, 17)


In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_mean = []
X_std = []

#code, jokey_w, dandivi, yeondivi, weight_diff, raw_weight, weight_added, prev1_velo, prev2_velo, distance, 
#3, 4,5,7,8,9,10,11,14
for i in range(X_train.shape[1]-2):
  X_mean.append(np.mean(X_train[:,i]))
  X_std.append(np.std(X_train[:,i]))
  print(i+1,"번째 변수의 X_mean:",'{:.3f}'.format(X_mean[i]),", X_std:",'{:.3f}'.format(X_std[i]))
  X_train[:,i] = (X_train[:,i]-X_mean[i])/X_std[i]
  X_test[:,i] = (X_test[:,i]-X_mean[i])/X_std[i]

train_size = len(X_train)
test_size = len(X_test)
size_x = len(X_train[0])
print("size_x : ",size_x)
print("train.shape: ",X_train.shape,"test.shape: ", X_test.shape)

# 설정하는 부분
training_epochs = 200
batch_size = 128

1 번째 변수의 X_mean: 32018.413 , X_std: 3566.007
2 번째 변수의 X_mean: 6.263 , X_std: 3.462
3 번째 변수의 X_mean: 3.782 , X_std: 1.058
4 번째 변수의 X_mean: 53.739 , X_std: 1.905
5 번째 변수의 X_mean: 39.975 , X_std: 55.572
6 번째 변수의 X_mean: 6.558 , X_std: 7.561
7 번째 변수의 X_mean: 0.218 , X_std: 0.413
8 번째 변수의 X_mean: 0.102 , X_std: 6.995
9 번째 변수의 X_mean: 471.736 , X_std: 27.113
10 번째 변수의 X_mean: 0.102 , X_std: 6.995
11 번째 변수의 X_mean: 15.509 , X_std: 0.439
12 번째 변수의 X_mean: 15.513 , X_std: 0.435
13 번째 변수의 X_mean: 0.349 , X_std: 0.477
14 번째 변수의 X_mean: 0.409 , X_std: 0.492
15 번째 변수의 X_mean: 1371.757 , X_std: 256.383
size_x :  17
train.shape:  (39603, 17) test.shape:  (16973, 17)


In [85]:
tf.reset_default_graph()

global_step = tf.Variable(0, trainable=False)
#learning_rate = tf.train.exponential_decay(0.01, global_step,
#                                           100000, 0.5, staircase=True)
learning_rate = 0.01
X = tf.placeholder(tf.float32, [None, size_x])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)
phase = tf.placeholder(tf.bool, name='phase')


W1 = tf.get_variable(shape=[size_x, 128], name='weight1', initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([128]))
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)
layer1 = tf.contrib.layers.batch_norm(layer1, center=True, scale=True, 
                                          is_training=phase)

W2 = tf.get_variable(shape=[128, 64], name='weight2', initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)
layer2 = tf.contrib.layers.batch_norm(layer2, center=True, scale=True, 
                                          is_training=phase)

W3 = tf.get_variable(shape=[64, 32], name='weight3', initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([32]))
layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
layer3 = tf.nn.dropout(layer3, keep_prob=keep_prob)
layer3 = tf.contrib.layers.batch_norm(layer3, center=True, scale=True, 
                                          is_training=phase)

W4 = tf.get_variable(shape=[32, 1], name='weight4', initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.nn.relu(tf.matmul(layer3, W4) + b4)
#hypothesis = tf.contrib.layers.batch_norm(hypothesis, center=True, scale=True, 
#                                        is_training=phase)


#cost = tf.reduce_mean(tf.square(hypothesis - Y))
cost = tf.losses.mean_squared_error(labels=Y, predictions=hypothesis)
#train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost,global_step=global_step)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_size / batch_size)

    for i in range(total_batch):
        batch_xs = X_train[i*batch_size:(i+1)*batch_size]
        batch_ys = y_train[i*batch_size:(i+1)*batch_size].reshape(batch_size,1)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7,phase:True}
        c, _,hyp= sess.run([cost, train,hypothesis], feed_dict=feed_dict)
        avg_cost += c/total_batch
        
    if epoch%10==0:
      #print(hyp);
      print('Epoch:', '%04d' % (epoch + 1), 'lr:', lr,' cost =', '{:.9f}'.format(avg_cost))
      #print(hyp)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


Epoch: 0001 lr: 0.01  cost = 1695.080446274
Epoch: 0011 lr: 0.01  cost = 69.758544212
Epoch: 0021 lr: 0.01  cost = 67.971862867
Epoch: 0031 lr: 0.01  cost = 63.248572174
Epoch: 0041 lr: 0.01  cost = 61.597216529
Epoch: 0051 lr: 0.01  cost = 61.180075655
Epoch: 0061 lr: 0.01  cost = 59.872639295
Epoch: 0071 lr: 0.01  cost = 60.019930441
Epoch: 0081 lr: 0.01  cost = 59.761217994
Epoch: 0091 lr: 0.01  cost = 59.837861632
Epoch: 0101 lr: 0.01  cost = 59.445219095
Epoch: 0111 lr: 0.01  cost = 59.175211379
Epoch: 0121 lr: 0.01  cost = 59.782808415
Epoch: 0131 lr: 0.01  cost = 58.459611325
Epoch: 0141 lr: 0.01  cost = 59.513608223
Epoch: 0151 lr: 0.01  cost = 58.904487091
Epoch: 0161 lr: 0.01  cost = 59.095380005
Epoch: 0171 lr: 0.01  cost = 59.210299736
Epoch: 0181 lr: 0.01  cost = 58.136632259
Epoch: 0191 lr: 0.01  cost = 58.282392847


In [86]:

y_test = y_test.reshape(-1,1)
'''
avg_cost = 0
total_batch = int(test_size / batch_size)

for i in range(total_batch):
    batch_xs = X_test[i*batch_size:(i+1)*batch_size]
    batch_ys = y_test[i*batch_size:(i+1)*batch_size].reshape(batch_size,1)
    feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:1.0,phase:False}
    c= sess.run(cost, feed_dict=feed_dict)
    avg_cost += c/total_batch

print(avg_cost)
'''
feed_dict = {X: X_test, Y: y_test, keep_prob:1.0, phase:False}
c, hyp = sess.run([cost,hypothesis], feed_dict=feed_dict)
print(c)
print(hyp)

404821870000.0
[[  3247.584]
 [     0.   ]
 [     0.   ]
 ...
 [     0.   ]
 [768750.4  ]
 [     0.   ]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
